In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datetime


In [2]:
list_of_dates = list(pd.date_range(start='1/1/1976', end='12/31/2000'))


In [3]:
def get_crossword_data(date):
    try:
        response = requests.get(f'https://www.xwordinfo.com/Crossword?date={date.month}/{date.day}/{date.year}')
        doc = BeautifulSoup(response.text)

        list_of_answers = []
        list_of_clues = []

        for across in doc.find_all("div",id="ACluesPan"):
            for clues in across.find("div", class_ = "numclue"):
                if clues.text.isnumeric() == False:
                    list_of_answers.append(clues.text.split(":")[1].strip())
                    list_of_clues.append(clues.text.split(":")[0].strip())

        for across in doc.find_all("div",id="DCluesPan"):
            for clues in across.find("div", class_ = "numclue"):
                if clues.text.isnumeric() == False:
                    list_of_answers.append(clues.text.split(":")[1].strip())
                    list_of_clues.append(clues.text.split(":")[0].strip())

        return pd.Series({'answers': list_of_answers, 'clues': list_of_clues})
    except:
        return pd.Series({'answers': [], 'clues': []})

In [ ]:
df = pd.DataFrame({
    'crossword_date': list_of_dates
})

results = df.crossword_date.apply(get_crossword_data)
df = df.join(results)
df = df.dropna()
df_answers_clues = df.set_index('crossword_date').apply(pd.Series.explode).reset_index()

df_answers_clues

In [ ]:
df_answers_clues['crossword_year'] = df_answers_clues['crossword_date'].dt.year
df_answers_clues.to_csv("crossword_2015.csv")

In [ ]:
df_answers_clues[df_answers_clues['clues'].str.contains('colloquially', regex=False) |df_answers_clues['clues'].str.contains('modern lingo', regex=False) | df_answers_clues['clues'].str.contains('slang', regex=False) | df_answers_clues['clues'].str.contains('informally', regex=False) | df_answers_clues['clues'].str.contains('slangily', regex=False)].head(50)